So things to think about 
1.  How to create validation set
    for each subject get a few samples for each class
2.  Scoring function
    log loss penalty is on giving low prediction to actual class so only clip low end of probabilities
3.  Which layers to train
    last two layers?

In [1]:
# open driver_imgs_list.csv
# read it into memory as a list of tuples
# for each subject, read each class into another array
# move 1/10th of those images into validation set


In [2]:
#so i want to do the notebook 1 stuff to load images
#but then instead of the fine tuning done in dogs in cats, I want to train the last convolutional layer (maybe 2 of them)
#since the last convolutional layer mainly learned birds and cats and single objects, and we really want lower level concepts
#i.e distracted person etc

In [1]:
import keras
import sys
sys.path.append('kernel')
import numpy as np
import os, sys
import shutil
%matplotlib inline

Using Theano backend.
Using cuDNN version 5110 on context None
Mapped name None to device cuda0: GeForce GTX 970 (0000:01:00.0)


In [28]:
path = "../data/statefarm/"

In [4]:
# open driver_imgs_list.csv
# read it into memory as a list of tuples
# for each subject, read each class into another array
# move 1/10th of those images into validation set
training_list = open(path + "driver_imgs_list.csv", "r")
headers = training_list.readline().rstrip().split(',')
print(headers)

['subject', 'classname', 'img']


In [6]:
#so i want a dictionary of subject -> class -> img name
training_data = dict()
for line in training_list:
    vals = line.rstrip().split(',')
    subj = vals[0]
    cls = vals[1]
    img = vals[2]
    
    if subj not in training_data:
        training_data[subj] = dict()
    if cls not in training_data[subj]:
        training_data[subj][cls] = list()
    training_data[subj][cls].append(img)
    

In [13]:
#for each subject
#for each class
#get the lenght of the list
#move one tenth of that list to validation and the rest to test
for subj,clses in training_data.iteritems():
    for cls,imgs in clses.iteritems():
        num_images = len(imgs)//10
        for val_image in imgs[:num_images]:
            postfix = str(cls) + "/" + val_image
            shutil.copy(path + "train/" + postfix,path + "train/val/" + postfix)
        for val_image in imgs[num_images:]:
            postfix = str(cls) + "/" + val_image
            shutil.copy(path + "train/" + postfix,path + "train/train/" + postfix)
    

In [5]:
import vgg16

In [23]:
# Import our class, and instantiate
import vgg16; reload(vgg16)
from vgg16 import Vgg16

In [24]:
m = Vgg16()

In [35]:
batch_size=8


In [36]:
batches = m.get_batches(path+'train/train/', batch_size=batch_size)
val_batches = m.get_batches(path+'train/val', batch_size=batch_size)

Found 20285 images belonging to 10 classes.
Found 2138 images belonging to 10 classes.


In [37]:
m.finetune(batches)


In [ ]:
m.fit(batches, val_batches, nb_epoch=1)

Epoch 1/1
 5704/20285 [=======>......................] - ETA: 378s - loss: 14.4325 - acc: 0.1033    

In [19]:
m.model.save_weights("VggModelV1.h5")

In [20]:

print("We reached end!")

In [22]:
#run with dogs and cats vgg
#run with one convolutional layer training
#run with two convolutional layers training
#keep adding until performance drops off
#if we are